In [39]:
import sys
from pathlib import Path

p = Path.cwd().resolve()
repo_root = next((parent for parent in [p] + list(p.parents) if (parent / ".git").exists()), None)
if repo_root is None:
    raise RuntimeError("Repo root not found. Open the repo folder in VS Code.")

sys.path.insert(0, str(repo_root))
print("Repo root:", repo_root)

Repo root: /Users/samiullah/Group_Project_Y3


In [40]:
import pandas as pd
import numpy as np

# Muni Codes

Gmina's change YOY
Doesn't matter as we simply aggregate 2021 commutes to county so use 2021 codes
Don't think we have any other gmina level figures

In [41]:
muni_codes = pd.read_excel(f"{str(repo_root)}/raw/00_codes/tablica_kts-teryt_2026v2.xls", sheet_name="Gminy 2021")
muni_codes.rename(columns={
    'Kod TERYT\nTERYT Code':'muni_code', # we should use teryt as code convention
    'Kod KTS\nKTS Code' : 'muni_kts',
    'Nazwa\nName' : 'muni_name'
}, inplace=True)

In [42]:
muni_codes["county_kts"] = (muni_codes["muni_kts"].astype(str).str[:11] + "000").astype(int)

In [43]:
muni_codes

,muni_code,muni_kts,muni_name,county_kts
0,201011,10030210101011,Bolesławiec (1),10030210101000
1,201022,10030210101022,Bolesławiec (2),10030210101000
2,201032,10030210101032,Gromadka (2),10030210101000
3,201043,10030210101043,Nowogrodziec (3),10030210101000
4,201044,10030210101044,Nowogrodziec - miasto (4),10030210101000
...,...,...,...,...
3813,3218054,10023216418054,Węgorzyno - miasto (4),10023216418000
3814,3218055,10023216418055,Węgorzyno - obszar wiejski (5),10023216418000
3815,3261011,10023216361011,Koszalin (1),10023216361000
3816,3262011,10023216562011,Szczecin (1),10023216562000


In [44]:
print('Num muni:', len(muni_codes), 'Num county:', muni_codes['county_kts'].nunique())

Num muni: 3818 Num county: 380


Correct

# County codes

In [45]:
county_codes = pd.read_excel(f"{str(repo_root)}/raw/00_codes/tablica_kts-teryt_2026v2.xls", sheet_name="Powiaty")

county_codes.rename(columns={
    'Kod TERYT\nTERYT Code':'county_code',
    'Kod KTS\nKTS Code': 'county_kts',
    'Nazwa\nName' : 'county_name'
}, inplace=True)


In [46]:
county_codes

,county_code,county_kts,county_name
0,201,10030210101000,Powiat bolesławiecki
1,202,10030210302000,Powiat dzierżoniowski
2,203,10030210203000,Powiat głogowski
3,204,10030210204000,Powiat górowski
4,205,10030210105000,Powiat jaworski
...,...,...,...
375,3217,10023216417000,Powiat wałecki
376,3218,10023216418000,Powiat łobeski
377,3261,10023216361000,Powiat m. Koszalin
378,3262,10023216562000,Powiat m. Szczecin


Join it back to muni_codes 

In [47]:
muni_codes = muni_codes.merge(county_codes, on='county_kts', how='left')

# Export

In [48]:
muni_codes.to_csv(f"{str(repo_root)}/cleaned/00_codes/muni_codes.csv", index=False)